<h2 style="font-weight: bold">Meets Bags of Popcorn</h2>

<h4>This is my first published notebook on Kaggle About NLP, So I decided ofc to take a look at the Bag of Words Meets Bags of Popcorn Compeition 😄😄<br><br>I will be doing a EDA of review texts, some Visualization and Pre-Processing. and finally modelling <br></h4>

* <h5 style="font-weight: 700">Your feedback is very welcome</h5>
* <h5 style="font-weight: 700">If you find this notebook useful, please don't forget to upvote it!</h5>

In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import re
import json
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup as bs
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
train = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', delimiter = "\t", encoding = 'utf-8')
test = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv.zip', delimiter = "\t", encoding = 'utf-8')

In [ ]:
sub=pd.read_csv('../input/word2vec-nlp-tutorial/sampleSubmission.csv')

In [ ]:
train.head()

#  **Exploratory Data Analysis**

In [ ]:
# Getting to know data
print(train.shape)
print(test.shape)

In [ ]:
train['length']=train['review'].apply(len)
train['length'].describe()

In [ ]:
#Sample review
print(train['review'][0])

In [ ]:
train_len=train['review'].apply(len)
test_len=test['review'].apply(len)

import matplotlib.pyplot as plt
import seaborn as sns
fig=plt.figure(figsize=(15,4))
fig.add_subplot(1,2,1)
sns.distplot((train_len),color='red')

fig.add_subplot(1,2,2)
sns.distplot((test_len),color='blue')

In [ ]:
train['word_n'] = train['review'].apply(lambda x : len(x.split(' ')))
test['word_n'] = test['review'].apply(lambda x : len(x.split(' ')))

fig=plt.figure(figsize=(15,4))
fig.add_subplot(1,2,1)
sns.distplot(train['word_n'],color='red')

fig.add_subplot(1,2,2)
sns.distplot(test['word_n'],color='blue')

In [ ]:
train['word_n'].describe()

* **Train and test set have a similar distribution of words in one review**
* **The mean words count is 233 and std is 173 words**

In [ ]:
cloud=WordCloud(width=800, height=600).generate(" ".join(train['review'])) # join function can help merge all words into one string. " " means space can be a sep between words.
plt.figure(figsize=(15,15))
plt.imshow(cloud)
plt.axis('off')

* **HTML tag like br should be removed**
* **words like movie or film are present in every review**

In [ ]:
fig, axe = plt.subplots(1,3, figsize=(23,5))
sns.countplot(train['sentiment'], ax=axe[0])
sns.boxenplot(x=train['sentiment'], y=train['length'], data=train, ax=axe[1])
sns.boxenplot(x=train['sentiment'], y=train['word_n'], data=train, ax=axe[2])

* **The distribution of sentiment is almost equal**
* **Lengh and number of words is similar across review but positive one tends to have more words**

# **Text Preprocessing**

In [ ]:
# English stopwords

stopwords=stopwords.words("english")
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
def cleaning(raw_text):
    # Removing HTML Tags
    html_removed_text=bs(raw_text).get_text()
    
    # Remove any non character
    character_only_text=re.sub("[^a-zA-Z]"," ",html_removed_text)
    
    # Lowercase and split
    lower_text=character_only_text.lower().split()
    
    # Get STOPWORDS and remove
    stop_remove_text=[i for i in lower_text if not i in stopwords]
    
    # Remove one character words
    lemma_removed_text=[word for word in stop_remove_text if len(word)>1]
    
    #Lemmatization
    lemma_removed_text=[wordnet_lemmatizer.lemmatize(word,'v') for word in stop_remove_text]
    
    
    return " ".join(lemma_removed_text)

In [ ]:
cleaning(train['review'][0])

In [ ]:
train['cleaned_review']=train['review'].apply(cleaning)

In [ ]:
train.head()

In [ ]:
test['review']=test['review'].apply(cleaning)